In [ ]:
# Installing dependencies
!pip install unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
Reason for being yanked: Error in the setup causing installation issues
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.11.1 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,!=4.57.0,<=4.57.2,>=4.51.3

In [ ]:
# Setting up the dataset
# Setting up the json file and the image folder
import json
converted_dataset = []
with open('/content/final_data.jsonl', 'r') as f:
    for line in f:
        converted_dataset.append(json.loads(line))
print(converted_dataset[0])


[{'role': 'user', 'content': [{'type': 'text', 'text': 'Can you identify the specific areas where the concentration of green-stained lymphatic vessels is the highest in the first image, and describe their relative position to the red-stained collagen areas?, having modality Microscopy Images of the body part Cell'}, {'type': 'image', 'image': 'images/pmc_10_0.jpg'}, {'type': 'image', 'image': 'images/pmc_10_1.jpg'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'In the first image, the highest concentration of green-stained lymphatic vessels appears centrally and slightly towards the upper region. These vessels are surrounded and somewhat intermingled with the red-stained collagen areas, suggesting close proximity and potential interactions between the lymphatic vessels and the surrounding collagen-rich tissue.'}]}]


In [ ]:
#  Model intilizing with unsloth
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit", # "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit" , "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit"
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Qwen2_Vl patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


In [ ]:
# PEFT-lora setup
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16*2,  # Recommended alpha == r at least
    lora_dropout = 0.05,
    bias = "none",
    random_state = 1200,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
print(model)

In [ ]:
# Traininer
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 1200,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,666 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 8,331,745,792 (0.48% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.431200
2,2.368000
3,2.443300
4,2.182600
5,2.188200
6,2.046400
7,1.762300
8,1.756500
9,1.765300
10,1.658200


In [ ]:
model.save_pretrained("Model_testing_v1")  # Local saving
tokenizer.save_pretrained("Model_testing_v1")

[]

In [ ]:
# Inference

FastVisionModel.for_inference(model) # Enable for inference!
# {"role": "assistant", "content": [{"type": "text", "text": "Of course. The echocardiogram shows that the right side of the heart is under significant strain. Specifically, the inferior vena cava, a large vein that carries blood from the lower body to the heart, is very dilated and there is abnormal blood flow. The right atrium, one of the heart's chambers, is also enlarged, which means it is dealing with more blood and pressure than normal. Additionally, there is a condition called tricuspid regurgitation, where the valve between the right atrium and right ventricle isn't closing properly, causing blood to flow backward into the atrium. The pressure measurements indicate that the pressure in the right side of the heart is quite high.\n\nThis situation suggests that the heart is having to work much harder to pump blood, particularly through the lungs, which might be due to conditions like pulmonary hypertension or other causes of right-sided heart failure. It's important to identify the underlying cause so we can determine the best treatment approach. This might involve medications to reduce the workload on the heart, procedures to address valve issues, or other treatments depending on the specific diagnosis and severity."}]}]


messages = [{"role": "user", "content": [{"type": "text", "text": "Doctor, can you explain what the echocardiogram shows? What does this mean for my loved one?, having modality Ultrasound of the body part Chest"},
{"type": "image", "image": "/content/pmc_10801_0.jpg"}, {"type": "image", "image": "/content/pmc_10801_1.jpg"}]} ]

image = ['/content/pmc_10801_0.jpg','/content/pmc_10801_1.jpg']
# messages = [
#       {"role": "user", "content": [
#           {"type": "image"},
#           {"type": "text", "text": instruction}
#       ]}
#   ]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
      image,
      input_text,
      add_special_tokens = False,
      return_tensors = "pt",
  ).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                    use_cache = True, temperature = 1, min_p = 0.1)

Certainly. The echocardiogram images show the heart's chambers and valves in detail. The first image (Fig. 1) shows a turbulent flow of blood through the tricuspid valve (TR Jet), which is a sign of tricuspid regurgitation. This means that some of the blood is leaking back into the right atrium instead of flowing forward into the right ventricle. The second image (Fig. 2) shows a similar pattern of blood flow through the pulmonary valve (PR), indicating pulmonary regurgitation. This means that some blood is leaking back into the right atrium from the pulmonary artery.


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
from bert_score import score
import torch

ground_truths = ["Of course. The echocardiogram shows that the right side of the heart is under significant strain. Specifically, the inferior vena cava, a large vein that carries blood from the lower body to the heart, is very dilated and there is abnormal blood flow. The right atrium, one of the heart's chambers, is also enlarged, which means it is dealing with more blood and pressure than normal. Additionally, there is a condition called tricuspid regurgitation, where the valve between the right atrium and right ventricle isn't closing properly, causing blood to flow backward into the atrium. The pressure measurements indicate that the pressure in the right side of the heart is quite high.\n\nThis situation suggests that the heart is having to work much harder to pump blood, particularly through the lungs, which might be due to conditions like pulmonary hypertension or other causes of right-sided heart failure. It's important to identify the underlying cause so we can determine the best treatment approach. This might involve medications to reduce the workload on the heart, procedures to address valve issues, or other treatments depending on the specific diagnosis and severity."]
responses = ["Certainly. The echocardiogram images show the heart's chambers and valves in detail. The first image (Fig. 1) shows a turbulent flow of blood through the tricuspid valve (TR Jet), which is a sign of tricuspid regurgitation. This means that some of the blood is leaking back into the right atrium instead of flowing forward into the right ventricle. The second image (Fig. 2) shows a similar pattern of blood flow through the pulmonary valve (PR), indicating pulmonary regurgitation. This means that some blood is leaking back into the right atrium from the pulmonary artery."]

P, R, F1 = score(responses, ground_truths, lang="en", verbose=True, nthreads=10)

print(
    f"""
Final Average BERTScore (Mean of all {len(P)} Samples):
Precision: {P.mean():.4f}
Recall: {R.mean():.4f}
F1 Score: {F1.mean():.4f}
"""
)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.29 seconds, 3.48 sentences/sec

Final Average BERTScore (Mean of all 1 Samples):
Precision: 0.8797
Recall: 0.8575
F1 Score: 0.8685



In [ ]:
model.push_to_hub("Haider584/lora_model", token = "hf_*******************") # Online saving
tokenizer.push_to_hub("Haider584/lora_model", token = "hf_*********************") # Online saving

README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 54.9kB /  162MB            

Saved model to https://huggingface.co/Haider584/lora_model


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mppx6u4oql/tokenizer.json:   0%|          | 26.1kB / 11.4MB            

In [ ]:
!zip /content/Model_testing_v1  Model_testing_v1

  adding: Model_testing_v1/ (stored 0%)
